# Split the data get from the WRDS database

In [2]:
import pandas as pd

# Load the data
file_path = 'return.csv' 
data = pd.read_csv(file_path)

# Splitting the file by PERMNO and saving into separate CSVs
for permno, group in data.groupby('PERMNO'):
    output_file = f'./stock/PERMNO_{permno}.csv' 
    group.to_csv(output_file, index=False)


# analyze the data, calculate beta, total risk, Jensen's alpha, and Sharpe ratio

In [3]:
import pandas as pd

# Load the provided file
file_path = './stock/PERMNO_10104.csv'
data_10104 = pd.read_csv(file_path)

# Display the first few rows of the data to understand its structure
data_10104.head()


,PERMNO,date,COMNAM,RET,sprtrn
0,10104,2011-12-30,ORACLE CORP,-0.181818,0.008533
1,10104,2012-01-31,ORACLE CORP,0.102144,0.043583
2,10104,2012-02-29,ORACLE CORP,0.037044,0.040589
3,10104,2012-03-30,ORACLE CORP,-0.003247,0.031332
4,10104,2012-04-30,ORACLE CORP,0.010288,-0.007497


In [6]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

def analyze_stock(PERMNO, risk_free_rate=0.02):
    PERMNO = str(PERMNO)
    path = f'./stock/PERMNO_{PERMNO}.csv'
    data = pd.read_csv(path)
    data['date'] = pd.to_datetime(data['date'])
    data['sprtrn'] = pd.to_numeric(data['sprtrn'], errors='coerce')
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data.dropna(subset=['sprtrn', 'RET'], inplace=True)
    monthly_risk_free_rate = (1 + risk_free_rate) ** (1 / 12) - 1
    slope, intercept, r_value, p_value, std_err = stats.linregress(data['sprtrn'], data['RET'])
    beta = slope
    total_risk = data['RET'].std()
    expected_return = monthly_risk_free_rate + beta * (data['sprtrn'] - monthly_risk_free_rate)
    jensens_alpha = (data['RET'] - expected_return).mean()
    excess_return = data['RET'] - monthly_risk_free_rate
    sharpe_ratio = excess_return.mean() / excess_return.std()
    
    # Adjusting the plot to have the x and y-axis intersect at (0,0)

    plt.figure(figsize=(10, 6))
    
    # Scatter plot of market returns vs stock returns
    plt.scatter(data['sprtrn'], data['RET'], color='blue', label='Data Points')
    
    # Regression line
    plt.plot(data['sprtrn'], intercept + slope * data['sprtrn'], color='red', label='SCL Line')
    
    plt.title(f'Security Characteristic Line (SCL) for PERMNO {PERMNO}')
    plt.xlabel('Market Returns (S&P 500)')
    plt.ylabel(f'Stock Returns (PERMNO {PERMNO})')
    
    # Set x and y axis to intersect at (0,0)
    plt.axhline(0, color='black',linewidth=0.8)
    plt.axvline(0, color='black',linewidth=0.8)
    
    plt.legend()
    plt.grid(True)
    
    plt.savefig(f'./stock/PERMNO_{PERMNO}.png')
    # plt.show()
    plt.close()

    
    return beta, total_risk, jensens_alpha, sharpe_ratio

analyze_stock(14702)

(1.6247510132530196,
 0.09756258140924692,
 0.00855579647172558,
 0.21586281010188596)

In [7]:
PERMNO_numbers = [10104,10107,13407,14593,14655,14702,15488,15579,17685,17972,18484,53613,59328,61241,75510,75754,76076,76201,77178,78975,84788,86580,88360,88362,89393,90215,90319,90857,92221,93002,93436]

with open('result.csv','w') as f:
    f.write('PERMNO,beta,total_risk,jensens_alpha,sharpe_ratio\n')
    for PERMNO in PERMNO_numbers:
        beta, total_risk, jensens_alpha, sharpe_ratio= analyze_stock(PERMNO)
        f.write(f'{PERMNO},{beta},{total_risk},{jensens_alpha},{sharpe_ratio}\n')
    